# dicom approach

In [63]:
from __future__ import print_function
import SimpleITK as sitk
import sys, time, os

data_directory = '/home/guilherme/Documents/noa/cidia19/data/spgc/dicom/P064'
out_directory = '/home/guilherme/Documents/noa/cidia19/data/spgc/dicom/P064-2'


series_IDs = sitk.ImageSeriesReader.GetGDCMSeriesIDs(data_directory)
if not series_IDs:
    print("ERROR: given directory \"" + data_directory + "\" does not contain a DICOM series.")
    
series_file_names = sitk.ImageSeriesReader.GetGDCMSeriesFileNames(data_directory, series_IDs[0])

series_reader = sitk.ImageSeriesReader()
series_reader.SetFileNames(series_file_names)

# Configure the reader to load all of the DICOM tags (public+private):
# By default tags are not loaded (saves time). By default if tags are loaded, the private tags are not loaded.
# We explicitly configure the reader to load tags, including the private ones.
series_reader.MetaDataDictionaryArrayUpdateOn()
series_reader.LoadPrivateTagsOn()
image3D = series_reader.Execute()

# Modify the image to 1mm spacing in Z
spacing, size = image3D.GetSpacing(), image3D.GetSize()
new_x_size, new_y_size = size[0], size[1]
new_z_size = int(size[2]*spacing[2]) #convert to 1mm
new_size = [new_x_size, new_y_size, new_z_size]
new_spacing = [old_sz*old_spc/new_sz for old_sz, old_spc, new_sz in zip(image3D.GetSize(), image3D.GetSpacing(), new_size)]
print(f"size:{size}, spacing:{spacing}, new_size:{new_size}, new_spacing:{new_spacing}")

interpolator_type = sitk.sitkLinear
first_image = sitk.Resample(image3D, new_size, sitk.Transform(), interpolator_type, image3D.GetOrigin(), 
                               new_spacing, image3D.GetDirection(), 0.0, image3D.GetPixelIDValue())



# Resize to 224, 224, Z/5
spacing, size = new_spacing, new_size
new_x_size, new_y_size, new_z_size = 224, 224, int(size[2]/5)
new_size = [new_x_size, new_y_size, new_z_size]
new_spacing = [old_sz*old_spc/new_sz for old_sz, old_spc, new_sz in zip(image3D.GetSize(), image3D.GetSpacing(), new_size)]
print(f"size:{size}, spacing:{spacing}, new_size:{new_size}, new_spacing:{new_spacing}")

interpolator_type = sitk.sitkLinear
filtered_image = sitk.Resample(first_image, new_size, sitk.Transform(), interpolator_type, first_image.GetOrigin(), 
                               new_spacing, first_image.GetDirection(), 0.0, first_image.GetPixelIDValue())
########################################

# Write the 3D image as a series
# IMPORTANT: There are many DICOM tags that need to be updated when you modify an
#            original image. This is a delicate opration and requires knowlege of
#            the DICOM standard. This example only modifies some. For a more complete
#            list of tags that need to be modified see:
#                           http://gdcm.sourceforge.net/wiki/index.php/Writing_DICOM

writer = sitk.ImageFileWriter()
# Use the study/series/frame of reference information given in the meta-data
# dictionary and not the automatically generated information from the file IO
writer.KeepOriginalImageUIDOn()

# Copy relevant tags from the original meta-data dictionary (private tags are
# also accessible).
tags_to_copy = ["0010|0010",  # Patient Name
                "0010|0020",  # Patient ID
                "0010|0030",  # Patient Birth Date
                "0020|000D",  # Study Instance UID, for machine consumption
                "0020|0010",  # Study ID, for human consumption
                "0008|0020",  # Study Date
                "0008|0030",  # Study Time
                "0008|0050",  # Accession Number
                "0008|0060"  # Modality
                ]

modification_time = time.strftime("%H%M%S")
modification_date = time.strftime("%Y%m%d")

# Copy some of the tags and add the relevant tags indicating the change.
# For the series instance UID (0020|000e), each of the components is a number,
# cannot start with zero, and separated by a '.' We create a unique series ID
# using the date and time.
# Tags of interest:
direction = filtered_image.GetDirection()
series_tag_values = [
                        (k, series_reader.GetMetaData(0, k))
                        for k in tags_to_copy
                        if series_reader.HasMetaDataKey(0, k)] + \
                    [("0008|0031", modification_time),  # Series Time
                     ("0008|0021", modification_date),  # Series Date
                     ("0008|0008", "DERIVED\\SECONDARY"),  # Image Type
                     ("0020|000e", "1.2.826.0.1.3680043.2.1125." +
                      modification_date + ".1" + modification_time),
                     # Series Instance UID
                     ("0020|0037",
                      '\\'.join(map(str, (int(direction[0]), int(direction[3]),
                                          int(direction[6]),
                                          # Image Orientation (Patient)
                                          int(direction[1]), int(direction[4]),
                                          int(direction[7]))))),
                     ("0008|103e",
                      series_reader.GetMetaData(0, "0008|103e")
                      + " Processed-SimpleITK")]  # Series Description

# filtered_image = image3D
# print(image3D)
# print(filtered_image)
for k in series_reader.GetMetaDataKeys(0):
#     print(k)
    v = series_reader.GetMetaData(0,k)
#     print(f"({k}) = = \"{v}\"")
    v = series_reader.GetMetaData(1,k)
#     print(f"({k}) = = \"{v}\"")

print('\n\n')


for i in range(filtered_image.GetDepth()):
    image_slice = filtered_image[:, :, i]
    # Tags shared by the series.
    for tag, value in series_tag_values:
        image_slice.SetMetaData(tag, value)
#         print(tag, image_slice.GetMetaData(tag))
    # Slice specific tags.
    #   Instance Creation Date
    image_slice.SetMetaData("0008|0012", time.strftime("%Y%m%d"))
    #   Instance Creation Time
    image_slice.SetMetaData("0008|0013", time.strftime("%H%M%S"))
    #   Image Position (Patient)
    image_slice.SetMetaData("0020|0032", '\\'.join(
        map(str, filtered_image.TransformIndexToPhysicalPoint((0, 0, i)))))
    #   Instace Number
    image_slice.SetMetaData("0020|0013", str(i))

    # Write to the output directory and add the extension dcm, to force writing
    # in DICOM format.
    writer.SetFileName(os.path.join(out_directory, str(i) + '.dcm'))
    writer.Execute(image_slice)

# print(filtered_image)
for k in filtered_image[:,:,0].GetMetaDataKeys():
    print(k)
    v = filtered_image.GetMetaData(k)
    print(f"({k}) = = \"{v}\"")
print('asdfsdf')

size:(512, 512, 144), spacing:(0.6484375, 0.6484375, 2.0), new_size:[512, 512, 288], new_spacing:[0.6484375, 0.6484375, 1.0]
size:[512, 512, 288], spacing:[0.6484375, 0.6484375, 1.0], new_size:[224, 224, 57], new_spacing:[1.4821428571428572, 1.4821428571428572, 5.052631578947368]



asdfsdf


In [65]:
import dicom2nifti
import dicom2nifti.settings as settings

settings.disable_validate_slice_increment()
# settings.enable_resampling()
# settings.set_resample_spline_interpolation_order(1)
# settings.set_resample_padding(-1000)
settings.set_gdcmconv_path("/home/guilherme/Documents/noa/cidia19/req_libs/GDCM-2.8.9-Linux-x86_64/bin/gdcmconv")

a = dicom2nifti.dicom_series_to_nifti(out_directory, 'P064.nii.gz')

In [66]:
a

{'NII_FILE': 'P064.nii.gz',
 'NII': <nibabel.nifti1.Nifti1Image at 0x7efe44042b70>,
 'MAX_SLICE_INCREMENT': 5.052631578947375}